# Region optimizer

Given a list of sources, finds a "good" way to observe them to minimize the slew time.

**Sebastian Kiehlmann, 2021-01-01, added option to limit the number of sources per region, larger regions are split.**

In [1]:
import sys, csv, os, random, math, time, pickle
import numpy
import matplotlib.pyplot as pyplot
import ipynb.fs.full.telescope_data as tel_data

import ipynb.fs.full.coordinate_utils as cu

In [4]:
# ------------------------------------------------------------------------------
# Class definition
# ------------------------------------------------------------------------------
class Region:
    """Stores the data for the sources on a given region and optimize
    the path. """

    # --------------------------------------------------------------------------
    def __init__(self, name, ra, dec, point, order):
        """ Initialize a source region. """

        #~~~~~~~~~~~~~~~~~~~~~~~~
        # Sources characteristics
        self.name = name
        self.ra = ra
        self.dec = dec
        self.point = point
        self.size = len(self.name)

        #~~~~~~~~~~~~~~
        # Initial order
        self.order = order

    # --------------------------------------------------------------------------
    def __len__(self):
        """Return the number of sources in this region.
        """
        return self.size

    # --------------------------------------------------------------------------
    def path_obstime(self, lst_start):
        """ Calculates the time given the lst for first source assuming
        telescope starts at that position

        The time it takes to move from the previous position to the first
        source in the region will be considered on the next layer of
        optimization

        The slew time is calculated using a zero step approximation, I could
        consider refining this in the future, but there is a penalty on
        computation time.

        NOTE:
        A sligthly modified version if this code is used on
        simulate_schedule_region() on simulate_schedules.py. If this code is
        modified that migth need some adjustements.

        """

        # current lst
        lst = lst_start

        # add slew and observing time for different sources
        for i in range(len(self.order) - 1):

            # get indices of sources
            a = self.order[i]
            b = self.order[i + 1]

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # time to observe first source and move on next one

            # get az/za for the two sources

            # Add observing time from source type, pointing or not
            lst += tel_data.obs_time(self.point[a])

            # First source at end of the observation
            za_a, az_a = cu.radec_zaaz(self.ra[a], self.dec[a], lst)

            # Estimate the slew time using the position of second source at
            # end of observation first source
            za_b, az_b = cu.radec_zaaz(self.ra[b], self.dec[b], lst)

            # add slew time
            lst += tel_data.slew_time(za_a, az_a, za_b, az_b)

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Add observation time for last source
        lst += tel_data.obs_time(self.point[self.order[-1]])

        self.obstime = lst - lst_start

        # return the path length
        return lst - lst_start


    # -------------------------------------------------------------------------
    def time_onsource(self):
        """ Returns the time spent on source for a given region
        """

        t_onsource = 0

        for type in self.point:
            t_onsource += tel_data.obs_time(type)

        return t_onsource


    # -------------------------------------------------------------------------
    def distance_to_pointing(self, lst_start):
        """ Get the distance on ZA, AZ from the source to pointing calibrator
        assumed to be the first source

        The first source on each region will be a pointing calibrator, but
        there might be more than one flux calibrator on a given region, and
        this has to be considered.

        NOTE:
        This is just to test roughly the distances.
        """

        # current lst
        lst = lst_start

        # distances array
        distance = []

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Solve the case with one source in the region
        if len(self.order) == 1:
            distance.append(0)

            return distance

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Solve the cases with more than 1 source

        # Coordinates of the first source on region that is always a pointing
        # calibrator. Get postion add end of observation
        point = self.order[0]
        t_obs = tel_data.obs_time(self.point[point])
        za_point, az_point = \
            cu.radec_zaaz(self.ra[point], self.dec[point], lst + t_obs)

        # add slew and observing time for different sources
        for i in range(len(self.order) - 1):

            # get indices of sources
            a = self.order[i]
            b = self.order[i + 1]

            #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # time to observe first source and move on next one

            # get az/za for the two sources

            # Add observing time from source type, pointing or not
            lst += tel_data.obs_time(self.point[a])

            # First source at end of the observation
            za_a, az_a = cu.radec_zaaz(self.ra[a], self.dec[a], lst)

            # Estimate the slew time using the position of second source at
            # end of observation first source
            za_b, az_b = cu.radec_zaaz(self.ra[b], self.dec[b], lst)

            # add slew time
            lst += tel_data.slew_time(za_a, az_a, za_b, az_b)

            # Get distance between this sources and pointing calibrator
            dis = cu.dis_zaaz(za_a, az_a, za_point, az_point)
            distance.append(dis)

        # Add observation time for last source
        lst += tel_data.obs_time(self.point[self.order[-1]])

        # Get its position at end of observation
        last = self.order[-1]
        za_a, az_a = cu.radec_zaaz(self.ra[last], self.dec[last], lst)

        # Get its distance to the pointing calibrator
        dis = cu.dis_zaaz(za_a, az_a, za_point, az_point)
        distance.append(dis)

        return distance


    # -------------------------------------------------------------------------
    def random_path(self):
        """ Pick a random observation path.

        The first source on each region is fixed, because it is a pointing
        calibrator
        """

        # Select the part we want to shuffle
        part_to_shuffle = self.order[1:]

        # Shuffle it
        random.shuffle(part_to_shuffle)

        # Save results
        self.order = [self.order[0]] + part_to_shuffle[:]


    # --------------------------------------------------------------------------
    def modify_path_random(self):
        """ Modify a given path slightly, swaping two sources

        The first source on each region is fixed, because it is a pointing
        calibrator

        NOTE:
        I should do something smarter on a second stage.
        """

        # select two random cities, by the indexes on self.order
        # First source is fixed
        sources = random.sample(range(1, len(self.name)), 2)

        # swap them
        source1 = self.order[sources[0]]
        self.order[sources[0]] = self.order[sources[1]]
        self.order[sources[1]] = source1


    # --------------------------------------------------------------------------
    def modify_path_random_section(self):
        """ Chose a random start and end and shuffle sources

        The first source on each region is fixed, because it is a pointing
        calibrator
        """

        # chose start and end index
        # First source is fixed
        sources = random.sample(1, range(len(self.name)), 2)
        start = min(sources)
        end = max(sources) + 1

        # select the section and shuffle elements
        order_part = self.order[start:end]
        random.shuffle(order_part)

        # apply changes
        self.order[start:end] = order_part


    # --------------------------------------------------------------------------
    def show_path(self, lst_start):
        """ Plot the telescope path given an starting lst using the
        current order.
        """

        # initial time
        lst = lst_start

        # Array for sources positions
        za = []
        az = []

        # Get coordinates considering t_obs and t_slew
        for i in range(len(self.order) - 1):

            # find indexes of consecutive sources
            a = self.order[i]
            b = self.order[i + 1]

            # Add observing time for the first source
            lst += tel_data.obs_time(self.point[a])

            # positons at end of observation
            za_a, az_a = cu.radec_zaaz(self.ra[a], self.dec[a], lst)
            za_b, az_b = cu.radec_zaaz(self.ra[b], self.dec[b], lst)

            # Store postion for first source
            za.append(za_a)
            az.append(az_a)

            # add slew time estimate
            lst += tel_data.slew_time(za_a, az_a, za_b, az_b)

        # Get position for last source at end of observation

        # Add observation time for this source
        lst += tel_data.obs_time(self.point[self.order[-1]])

        # calculate position of last source and add to arrays
        a = self.order[-1]
        za_a, az_a = cu.radec_zaaz(self.ra[a], self.dec[a], lst)

        za.append(za_a)
        az.append(az_a)

        # plot the path on telescope coordinates
        pyplot.plot(az, za)
        pyplot.show()


    # --------------------------------------------------------------------------
    def brute_force_optimize(self, n_iter, lst_start):
        """Ramdomly sample different paths and choose the best
        n_iter is the number of iterations.
        lst_start lst for begining of observation
        """

        # starting condition. path_obstime needs to be calculated
        # since it depends on lst_start
        minimum_obstime = self.path_obstime(lst_start)

        print('Initial path length = ', minimum_obstime)

        if len(self.ra) <= 2:
            print( 'No optimization necessary')
            print( 'Final path length = ', minimum_obstime)
            return

        # search for best path
        for i in range(n_iter):

            # saves the initial order and chose a random one
            order_i = self.order[:]
            self.modify_path_random()
            self.path_obstime(lst_start)

            # if length increases, revert changes
            if self.obstime > minimum_obstime:
                self.order = order_i

            else:
                minimum_obstime = self.obstime

        print('Final path length = ', minimum_obstime)
    
    # --------------------------------------------------------------------------
    def nearest_neighbor(self, lst_start, start_source=0):
        """ Construct the NN path"""

        print('Nearest neighbour solution')
        print('Initial length = ', self.path_obstime(lst_start))

        # initialize the lst
        lst  = lst_start

        # sources to include in path
        sources_left = range(len(self.name))

        # get current source and delete from list of left sources
        current_source = sources_left.pop(start_source)

        # source order
        order_nn = [current_source]

        # loop through all the sources not in the path
        for i in range(len(self.name) - 1):

            #
            # get slew time from current source to the rest
            #

            # get observing time from source type
            t_obs = tel_data.obs_time(self.point[current_source])

            # get coordinates of current source at end of observation
            a = current_source
            za_a, az_a = cu.radec_zaaz(self.ra[a], self.dec[a], lst + t_obs)

            # get source with smaller slew time from source a
            min_slew = -1
            closest_source_index = -1
            for j in range(len(sources_left)):

                # get the index of the test source
                b = sources_left[j]

                # Estimate the slew time using the position of second source at
                # end of observation first source
                za_b, az_b = cu.radec_zaaz(self.ra[b], self.dec[b], lst + t_obs)

                # slew time
                t_slew = tel_data.slew_time(za_a, az_a, za_b, az_b)

                # check if the slew time is smaller than minimum
                if t_slew < min_slew:
                    min_slew = t_slew
                    closest_source_index = j

            # find the closest source and delete it from left sources list
            current_source = sources_left.pop(closest_source_index)

            # add it to the path
            order_nn.append(current_source)

            # update the lst
            lst += t_obs + t_slew

        # modify order of sources
        self.order = order_nn


        print('Final length = ', self.path_obstime(lst_start))


    # --------------------------------------------------------------------------
    def metropolis_filter(self, delta, T):
        """ Decides which modifications we keep"""

        # Ramdom number between [0,1]
        x = random.random()

        # Apply filter
        if delta < 0 or x < math.exp(-delta / T):
            accept = True
        else:
            accept = False

        return accept


    # --------------------------------------------------------------------------
    def sa_optimize(self, lst_start):
        """ Simulated annealing optimization"""

        # Initial temperature
        initial_obstime = self.path_obstime(lst_start)
        Ti = initial_obstime
        T = initial_obstime

        # number of repetitions per temperature step
        r = 10

        print('Initial length = ', initial_obstime)

        if len(self.ra) <= 2:
            print('No optimization necessary')
            print('Final path length = ', initial_obstime)
            return

        # Store the different path lengths accepted
        sol_obstime = [self.obstime]

        while T > Ti*0.01:

            for i in range(r):
                # Store current path and modify the starting path
                order_i = self.order[:]
                obstime_i = self.obstime
                self.modify_path_random()


                # Length difference between paths
                delta = self.path_obstime(lst_start) - obstime_i

                # Apply Metropolis filter
                if not self.metropolis_filter(delta, T):
                    # New solution is rejected, go back
                    self.order = order_i
                    # recalculate the obstime to go back to initial case
                    self.path_obstime(lst_start)
                # Output the current solution
                sol_obstime.append(self.obstime)

            # Update temperature
            T = 0.9 *  T

        print('Final length = ', self.obstime)
        print('Sleweing time percentage = ',\
            (self.obstime - self.time_onsource()) / self.obstime)

        #pyplot.plot(sol_obstime)
        #pyplot.show()
        #self.show_path()


    # --------------------------------------------------------------------------
    def sa_optimize_alt_cooling(self, lst_start):
        """ Simulated annealing optimization"""

        t0 = time.time()
        
        # Initial obstime
        initial_obstime = self.path_obstime(lst_start)

        # Number of sources
        n_sources = len(self.name)

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Initial temperature
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # number of tries
        Ntry = 100

        # Used to get variance
        sum_obstime = 0
        sum2_obstime = 0

        # Try a few random paths to get a sense for rms
        for i in range(Ntry):

            obstime = self.path_obstime(lst_start)
            self.random_path()

            sum_obstime += obstime
            sum2_obstime += obstime ** 2

        sum_obstime = sum_obstime / Ntry
        sum2_obstime = sum2_obstime / Ntry
        rms_obstime = math.sqrt(abs(sum2_obstime - sum_obstime ** 2))

        # initial temperature
        Ti = 3 * rms_obstime
        T = Ti

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Algorithm parameters
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        # number of repetitions per temperature step
        r = 100

        # number of successes and failures per temperature step
        n_succ = 0
        n_fail = 0

        print('Initial length = ', initial_obstime)

        if len(self.ra) <= 2:
            print( 'No optimization necessary')
            print( 'Final path length = ', initial_obstime)
            return

        # Store the different path lengths accepted
        sol_obstime = [self.obstime]

        while T > Ti*0.01:

            for i in range(r):
                # Store current path and modify the starting path
                order_i = self.order[:]
                obstime_i = self.obstime
                self.modify_path_random()


                # Length difference between paths
                delta = self.path_obstime(lst_start) - obstime_i

                # Apply Metropolis filter
                if not self.metropolis_filter(delta, T):

                    # New solution is rejected, go back
                    self.order = order_i
                    # recalculate the obstime to go back to initial case
                    self.path_obstime(lst_start)

                    n_fail += 1

                else:

                    # New solution is accepted
                    n_succ += 1

                # Output the current solution
                sol_obstime.append(self.obstime)

            # Update temperature if needed
            if n_succ > 0 or n_fail > 0:
                T = 0.9 *  T
                n_succ = 0
                n_fail = 0

        print( 'Final length = ', self.obstime)
        print('time spent on source for a given region = ', self.time_onsource())
        print( 'slewing time percentage = ',\
            (self.obstime - self.time_onsource()) / self.obstime)
        #print( 'el orden es: ', self.order)
        
        #pyplot.plot(sol_obstime)
        #pyplot.show()
        #self.show_path(lst_start)
        
        minimum_obstime = self.obstime
        order_opt = self.order
        slewing_time = (minimum_obstime - self.time_onsource()) / minimum_obstime
        time_onsource = self.time_onsource()
    
        tf = time.time()
        time_taken = tf-t0
        
        return minimum_obstime, time_onsource, slewing_time, order_opt, time_taken


    # --------------------------------------------------------------------------
    def exact_solution(self, lst_start):
        """ Exact solution by enumeration of all the possible cases

        - Preserve position of first source. This is because this source is a
        pointing calibrator that has to be observed before all the others

        Use only for cases with small number of sources. The number of
        possible paths is n! so it grows very fast.
        """
        t0 = time.time()
        
        def all_perms(str):
            """ Finds all the permutations and creates iterator
            Works with strings and lists"""

            if len(str) <=1:
                yield str
            else:
                for perm in all_perms(str[1:]):
                    for i in range(len(perm)+1):
                        yield perm[:i] + str[0:1] + perm[i:]
            return


        # starting condition
        minimum_obstime = self.path_obstime(lst_start)
        order_opt = self.order
        first_source_index = self.order[0]

        print( 'Exact solution')
        print( 'Initial path obstime = ', minimum_obstime)

        # search for best path, goes through all the permutations
        for p_p in all_perms(self.order[1:]):

            # Calculate obstime for this particular permutation
            p = [first_source_index] + p_p
            self.order = p
            self.path_obstime(lst_start)

            # if obstime is reduced we have new optimum
            if self.obstime < minimum_obstime:
                minimum_obstime = self.obstime
                order_opt = self.order

        # Use the optimum order to get the minimum_obstime
        self.order = order_opt
        self.path_obstime(lst_start)

        print( 'Final path length = ', minimum_obstime)
        print( 'slewing time percentage = ',\
            (self.obstime -  self.time_onsource()) / self.obstime)
        
        slewing_time = (minimum_obstime - self.time_onsource()) / minimum_obstime
        time_onsource = self.time_onsource()
    
        tf = time.time()
        time_taken = tf-t0
        
        return minimum_obstime, time_onsource, slewing_time, order_opt, time_taken



    # --------------------------------------------------------------------------
    def split(self, region_source_limit, lst_start):
        """Return list of regions with fewer sources than the limit. The initial
        source order is preserved. Each new region uses the pointing calibrator
        of the initial region. All new regions will be approximately of the same
        size.

        Note: run split() only after region optimization!
        """

        n_new = int(math.ceil(self.size *1. / region_source_limit))
        n_split = int(math.ceil(self.size * 1. / n_new)) - 1 # minus 1 for the pointing calibrator

        print( 'Region is split into %d regions' % n_new)
        new_regions = []
        # order lists:
        order = numpy.argsort(self.order)
        self.name = [self.name[i] for i in order]
        self.ra = [self.ra[i] for i in order]
        self.dec = [self.dec[i] for i in order]
        self.point = [self.point[i] for i in order]
        self.order = [self.order[i] for i in order]

        # get pointing calibrator and remove it from lists:
        i_pcal = numpy.argmax(self.point)
        pcal_name = self.name[i_pcal]
        pcal_ra = self.ra[i_pcal]
        pcal_dec = self.dec[i_pcal]
        self.name.pop(i_pcal)
        self.ra.pop(i_pcal)
        self.dec.pop(i_pcal)

        # perform splits:
        for i in range(n_new):
            sources_name = self.name[n_split*i:n_split*(i+1)]
            sources_ra = self.ra[n_split*i:n_split*(i+1)]
            sources_dec = self.dec[n_split*i:n_split*(i+1)]
            name = [pcal_name] + sources_name
            ra = [pcal_ra] + sources_ra
            dec = [pcal_dec] + sources_dec
            point = [0]*(n_split+1)
            point[0] = 1
            order = [j for j in range(n_split+1)]
            region = Region(name, ra, dec, point, order)
            region.path_obstime(lst_start)
            new_regions.append(region)

            print( 'Sub-region ', i)
            print( 'Final length = ', region.obstime)
            print( 'slewing time percentage = ', \
                    (region.obstime - region.time_onsource()) / region.obstime)

        return new_regions

### Auxiliary functions

In [3]:
def load_sources(sources_file):
    """ Load sources data structure from pickle file
    """

    file = open(sources_file, 'r')

    sources = pickle.load(file)

    file.close()

    return sources

In [4]:
def create_region_objects(sources_file):
    """ Using the output file created with sky_regions creates a sources
    data structure and an list of regions with region coordinates, list of
    sources and the order of the observations. For the order starts with
    1,2,3,..,N
    """

    #~~~~~~~~~~~~~~~~~~~~
    # load data from file
    sources = load_sources(sources_file)

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # find maximum region number
    max_region = -1
    for source in sources:

        if sources[source]['region'] > max_region:
            max_region = sources[source]['region']

    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Create an array of regions. Each object would be a region with
    # at least one source
    #

    regions = []

    # for all the regions upto maximum found on sources_file
    for i in range(max_region + 1):

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # find all the sources on that region
        sources_region = [name for name in sources if \
                              sources[name]['region'] == i]

        # If empty region go to next one
        if len(sources_region) == 0:
            continue

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Get the center of the region as the coordinate of the first
        # pointing source

        # find first pointing source on that region
        first_point_source = [name for name in sources_region if \
                                sources[name]['pointing'] == 1][0]

        # coordinates for center of region
        ra_r = sources[first_point_source]['ra']
        dec_r = sources[first_point_source]['dec']

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # Initial guess for source order.
        # First source is first pointing source

        order = range(len(sources_region))

        # find index of first pointing source
        index_first = [j for j in range(len(sources_region)) if \
                           sources[sources_region[j]]['pointing'] == 1][0]

        # Put this as the first and swap with other source
        index_other = order[0]
        order[index_first] = index_other
        order[0] = index_first

        #~~~~~~~~~~~~~~~~~~~~~~
        # create a region entry
        region = {'number': i,
                  'ra': ra_r,
                  'dec': dec_r,
                  'sources': sources_region,
                  'order': order}

        # add it to the regions list
        regions.append(region)

    return sources, regions

In [5]:
def sources_observability(sources, zmin=tel_data.zmin_s, zmax=tel_data.zmax_s,
                          dec_min=tel_data.dec_min,
                          zmax_low_dec=tel_data.zmax_low_dec_s):
    """ Creates a new key for each source that contains the observability
    ranges in the format

    sources[name]['obs_range'] = [[lst_l, lst_u], [lst_l, lst_u]]

    This information will be use on the schedule generation to avoid trying to
    observe sources at lsts when it is not observable.
    The idea is to skip the source if out of these ranges.

    notes
    -----
    - Nov 20, 2010: A few MOJAVE sources with very low declination were added
    to the schedule. In order to observe them a wider observing window was
    required and the different zmax for low declination sources was added.
    """

    # Add obs_range to every source
    for name in sources:
        ra = sources[name]['ra']
        dec = sources[name]['dec']

        # Find appropriate zenith angle limit
        if dec < dec_min:
            zmax_lim = zmax_low_dec

        else:
            zmax_lim = zmax


        sources[name]['obs_range'] = cu.source_obsrange(ra, dec, zmin, zmax_lim)

    return sources


In [6]:
def regions_observability(sources, regions):
    """
    Creates a new key for each region that contains the observability
    using the same format as for sources

    The region observability is the intersection of the observability for all
    sources.

    NOTE:
    I should project the observability window for all the sources to the time
    of the first one, but first I will see how this simple strategy works.
    """
    
    # Add obs_range to every region
    for region in regions:

        # Take the intersection of obs_ranges for all sources in region
        obs_range_sources = []
        for source in region['sources']:

            # get observing range for source and added it to the list
            obs_range_source = sources[source]['obs_range']
            obs_range_sources.append(obs_range_source)

        # calculate intersection
        obs_range = cu.intersect_obs_ranges(obs_range_sources, t_resol=1.0)

        # save resulting observing range for region
        region['obs_range'] = obs_range

    return regions


In [1]:
def optimize_all_regions(sources, regions, n_exact=8, region_source_limit=None):
    """ Optimize the observing time for the regions assuming they are observed
    2 hours before transit

    input
    -----
    sources
    regions
    n_exact=8                  Solve exactly for regions with equal or less sources
    region_source_limit=None   Regions with more sources than limit are split

    output
    ------


    notes
    -----
    In the final version I have to use a real starting lst. This is tricky
    since it could depend on the organization of the regions. I can use the
    solution at a fiducial hour angle and perturb that on the real case to get
    the best order, or at least check that at the real start lst the time for
    the observation is not very different from the initial estimate.
    """

    # total observation time
    total_time = 0

    # List of distances to pointing calibrator
    distances = []
    new_regions = []
    data = []

    
    #print(i)
    
    # loop through the regions
    for i in range(len(regions)):
        
        # select the appropiate region
        region =  regions[i]

        # get region information to initialize object
        region_number = region['number']
        names = region['sources']
        order = region['order']

        # get source information from sources dictionary
        ra = [sources[name]['ra'] for name in names]
        dec = [sources[name]['dec'] for name in names]
        point = [sources[name]['pointing'] for name in names]

        # create the object
        R = Region(names, ra, dec, point, order)

        # Optimize the region.
        # USE DIFFERENT METHODS TO TEST
        n_iter = 100
        lst_start = (region['ra'] / 15.0 - 2.0) % 24.0

        t0 = time.time()

        print( '---------------------------------------')
        print( 'Optimizing region ', region_number,\
            'n_sources = ', len(regions[i]['order']))

        if len(order) > n_exact:
            #R.sa_optimize(lst_start)
            
            minimum_obstime, time_onsource, slewing_time, order_opt, time_taken = R.sa_optimize_alt_cooling(lst_start)
            distances += R.distance_to_pointing(lst_start)
            
            data_region = {
                'region number': region_number, 'obstime': minimum_obstime, 
                'time onsource': time_onsource, 'slewing time': slewing_time,
                'order_opt': order_opt, 'time taken': time_taken}
            data.append(data_region)
            
            #R.genetic_algorithm(lst_start, region_number)
            #R.show_path(lst_start)
            #R.nearest_neighbor(lst_start, start_source=0)
            #R.SwapAlgorithm_main(lst_start)
        else:
            minimum_obstime, time_onsource, slewing_time, order_opt, time_taken = R.exact_solution(lst_start)
            
            data_region = {
                'region number': region_number, 'obstime': minimum_obstime, 
                'time onsource': time_onsource, 'slewing time': slewing_time,
                'order_opt': order_opt, 'time taken': time_taken}
            data.append(data_region)
            
            #R.nearest_neighbor(lst_start, start_source=0)
            #R.show_path(lst_start)

        #R.brute_force_optimize(n_iter, lst_start)
        

        # no splitting required:
        if region_source_limit is None or R.size <= region_source_limit:
            # Add to total time
            total_time += R.obstime

            new_region = {
                    'order': R.order, 'sources': R.name,
                    'number': '%i' % region_number, 'ra': region['ra'],
                    'dec': region['dec'], 'lst_start': lst_start,
                    'obstime': R.obstime}
            new_regions.append(new_region)

        # split region:
        else:
            print( 'Region exceeds source observation limit')
            split_regions = R.split(region_source_limit, lst_start)
            for (R, char) in zip(split_regions, 'abcdefghijklmnopqrstuvwxyz'):
                new_region = {
                        'order': R.order, 'sources': R.name,
                        'number': '%i%s' % (region_number, char),
                        'ra': region['ra'],
                        'dec': region['dec'], 'lst_start': lst_start,
                        'obstime': R.obstime}
                new_regions.append(new_region)
                total_time += R.obstime

        # TODO: What are 'distances' used for?

    print( '----------------------------------')
    print( 'Total observing time ', total_time)

    return sources, new_regions, data


In [8]:
def solution_stats(regions, plot_path):
    """ Compare results and upper bounds bassed on telescope model
    """

    # get observing time and number of sources per region
    obs_time = [region['obstime'] for region in regions]
    n_sources = [len(region['sources']) for region in regions]

    # get upper bounds
    n = numpy.arange(1, max(n_sources) + 2)

    # calculate for different average distance between sources
    t_15 = tel_data.time_dopoint + tel_data.time_doflux * n +\
        (60./3600.)*(15.0/15.0)*(n - 1)
    t_10 = tel_data.time_dopoint + tel_data.time_doflux * n +\
        (60./3600.)*(10.0/15.0)*(n - 1)
    t_5 = tel_data.time_dopoint + tel_data.time_doflux * n +\
        (60./3600.)*(5.0/15.0)*(n - 1)
    t_0 = tel_data.time_dopoint + tel_data.time_doflux * n +\
        (60./3600.)*(0.0/15.0)*(n - 1)

    # plot data and upper bounds
    pyplot.clf()
    pyplot.plot(n_sources, obs_time, '.', label='region observing time')
    pyplot.plot(n, t_15, label='average distance 15 deg')
    pyplot.plot(n, t_10, label='average distance 10 deg')
    pyplot.plot(n, t_5, label='average distance 5 deg')
    pyplot.plot(n, t_0, label='lower limit')
    pyplot.legend(loc=0, frameon=False)
    pyplot.title('Observing time versus upper bounds')
    pyplot.xlabel('number of sources')
    pyplot.ylabel('obs time hours')
    pyplot.savefig(plot_path)

    return

In [9]:
def save_variables(sources, regions, savefile):
    """ Save variables in a file for later use
    """

    file = open(savefile, 'w')

    pickle.dump(sources, file)
    pickle.dump(regions, file)

    file.close()

    return


# Main program

In [10]:
"""
sources_file = 'divide_sky/sources_test.dat'
sources, regions = create_region_objects(sources_file)

sources, regions, distances = optimize_all_regions(sources, regions)
sources = sources_observability(sources)
regions = regions_observability(regions)

save_variables(sources, regions, 'region_optimizer_results.dat')

# Histogram of distances from source to calibrators
pyplot.hist(distances, bins=20)
pyplot.show()
"""

"\nsources_file = 'divide_sky/sources_test.dat'\nsources, regions = create_region_objects(sources_file)\n\nsources, regions, distances = optimize_all_regions(sources, regions)\nsources = sources_observability(sources)\nregions = regions_observability(regions)\n\nsave_variables(sources, regions, 'region_optimizer_results.dat')\n\n# Histogram of distances from source to calibrators\npyplot.hist(distances, bins=20)\npyplot.show()\n"